In [26]:

import numpy as np
import glob
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow import keras
import pandas as pd
import random

print("Hello World")
print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))


Hello World
GPU name:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [27]:

# Now we wish to load in the data. For now this will just be in the local directory

data_dir = "D:\\POLYGON_TRAINING\\"
file_list = glob.glob(data_dir + "*.npy" )
random.shuffle(file_list)
file_list = file_list[:1400]
print(str(len(file_list)) + " files found")
batch_size = 200
image_size = 128


1400 files found


In [28]:
def resblock(x, kernelsize, filters):
    fx = layers.Conv2D(filters, kernelsize, activation='relu', padding='same')(x)
    fx = layers.BatchNormalization()(fx)
    fx = layers.Conv2D(filters, kernelsize, padding='same')(fx)
    out = layers.Add()([x,fx])
    out = layers.ReLU()(out)
    out = layers.BatchNormalization()(out)
    return out

In [29]:
#ResUNet Architecture:

inputs = keras.Input(shape=(128, 128, 16))
x = layers.Conv2D(32, 3, activation="relu",padding = "same")(inputs)
x = layers.Conv2D(64, 3, activation="relu",padding = "same")(x)
x = layers.BatchNormalization()(x)
block_1_output = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(128, 3, activation="relu", padding="same")(block_1_output)
x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
block_2_output = layers.concatenate([x, block_1_output])

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_2_output)
x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
block_3_output = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(128, 3, activation="relu", padding="same")(block_3_output)
x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
x = layers.Dropout(0.4)(x)
x = layers.BatchNormalization()(x)
block_4_output = layers.concatenate([x, block_3_output])


x = layers.Conv2D(128, 3, activation="relu", padding="same")(block_4_output)
x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
block_5_output = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(128, 3, activation="relu", padding="same")(block_5_output)
x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
x = layers.Dropout(0.4)(x)
x = layers.BatchNormalization()(x)
block_6_output = layers.concatenate([x, block_5_output])


x = layers.Conv2D(256, 3, activation="relu", padding="same")(block_6_output)
x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
block_5_output = layers.MaxPooling2D(2)(x)
block_5_output = layers.UpSampling2D(2)(block_5_output)

x = layers.UpSampling2D(2, interpolation = 'bilinear')(block_5_output)
x = layers.BatchNormalization()(x)

x = layers.UpSampling2D(2, interpolation = 'bilinear')(x)
x = layers.BatchNormalization()(x)

x = layers.UpSampling2D(2, interpolation = 'bilinear')(x)
x = layers.BatchNormalization()(x)

outputs = layers.Conv2D(2, 3, activation="sigmoid", padding = "same")(x)

model = keras.Model(inputs, outputs, name="resunet1")
model.summary()

Model: "resunet1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 128, 128, 16 0                                            
__________________________________________________________________________________________________
conv2d_141 (Conv2D)             (None, 128, 128, 32) 4640        input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_142 (Conv2D)             (None, 128, 128, 64) 18496       conv2d_141[0][0]                 
__________________________________________________________________________________________________
batch_normalization_79 (BatchNo (None, 128, 128, 64) 256         conv2d_142[0][0]                 
___________________________________________________________________________________________

In [30]:
#Loss function
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = tf.compat.v1.layers.flatten(y_true)
    y_pred_f = tf.compat.v1.layers.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


In [31]:
adam = keras.optimizers.Adam()
model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef])
model.summary()


Model: "resunet1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 128, 128, 16 0                                            
__________________________________________________________________________________________________
conv2d_141 (Conv2D)             (None, 128, 128, 32) 4640        input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_142 (Conv2D)             (None, 128, 128, 64) 18496       conv2d_141[0][0]                 
__________________________________________________________________________________________________
batch_normalization_79 (BatchNo (None, 128, 128, 64) 256         conv2d_142[0][0]                 
___________________________________________________________________________________________

In [32]:
#Let's try running it


sat_data_list = np.zeros((batch_size,128,128,16))
poly_data_list = np.zeros((batch_size,128,128,2))

for j in range(int(len(file_list)/batch_size)):
    print("processing batch " + str(j+1) + " of " + str(len(file_list)/batch_size))
    for idx, file in enumerate(file_list[batch_size*j:batch_size*(j+1)]):
        file_data = np.load(file)

        # Seperate out the actual data from the desired output and replace NaNs with 0
        sat_data = np.nan_to_num(file_data[192:320,192:320,:16])
        poly_data = np.dstack((np.nan_to_num(file_data[192:320,192:320,16]),1-(np.nan_to_num(file_data[192:320,192:320,16]))))

        sat_data_normalised = tf.keras.utils.normalize(sat_data,axis = -1)
        poly_data_normalised = tf.keras.utils.normalize(poly_data)
        # This will ideally need normalising before going any further. poly_data should already be 0 or 1

        #The Neural network requires an extra dimension which is the concatenation of all the images
        sat_data_list[idx] = sat_data_normalised[:,:,:]
        poly_data_list[idx] = poly_data_normalised[:,:,:]
    # Ok now we have the data. Let's rename them X and Y for brevity

    X = sat_data_list
    Y = poly_data_list

    model.fit(X,Y, epochs = 12, batch_size = 20, validation_split = 0.1,verbose = 1)





processing batch 1 of 7.0
Epoch 1/12


D:\Miniconda\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
D:\Miniconda\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:2183: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
D:\Miniconda\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:1331: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


9/9 [==============================] - 32s 304ms/step - loss: 0.3351 - dice_coef: 0.6649 - val_loss: 0.9264 - val_dice_coef: 0.0736
Epoch 2/12
9/9 [==============================] - 1s 88ms/step - loss: 0.1861 - dice_coef: 0.8139 - val_loss: 0.1667 - val_dice_coef: 0.8333

In [34]:
#Ok let's see what this predicts

test_dir = "D:\\POLYGON_VALIDATION\\"
file_list_test = glob.glob(test_dir + "*.npy" )
print(str(len(file_list_test)) + " files found for testing")

# Now we use numpy.load to open the files

sat_data_list_test = np.zeros((len(file_list_test),128,128,16))
poly_data_list_test = np.zeros((len(file_list_test),128,128,2))

for idx, file in enumerate(file_list_test):
    file_data = np.load(file)

    # Seperate out the actual data from the desired output and replace NaNs with 0
    sat_data_test = np.nan_to_num(file_data[192:320,192:320,:16])
    poly_data_test = np.dstack((np.nan_to_num(file_data[192:320,192:320,16]),1-(np.nan_to_num(file_data[192:320,192:320,16]))))

    sat_data_normalised_test = tf.keras.utils.normalize(sat_data_test,axis = -1)
    poly_data_normalised_test = tf.keras.utils.normalize(poly_data_test)
    # This will ideally need normalising before going any further. poly_data should already be 0 or 1

    #The Neural network requires an extra dimension which is the concatenation of all the images
    sat_data_list_test[idx] = sat_data_normalised_test
    poly_data_list_test[idx] = poly_data_normalised_test
# Ok now we have the data. Let's rename them X and Y for brevity

X_test = sat_data_list_test
Y_test = poly_data_list_test

print("About the make a prediction. Jeronimo!! ")

prediction = model.predict(X_test)

print("Prediction generated. ")
np.ndarray.view(prediction)
for i in range(0,6):
    plt.imsave("prediction_" + str(i) + ".png",prediction[i,:,:,0] , cmap = "binary")
    plt.imsave("true_" + str(i) + ".png",Y_test[i,:,:,0] , cmap = "binary")
    plt.imsave("IR_" + str(i) + ".png",X_test[i,:,:,15] , cmap = "binary")


7 files found for testing
About the make a prediction. Jeronimo!! 
Prediction generated. 
